### Ergodic Metric Using iLQR

In [41]:
#import necessary packages 
import numpy as np
from numpy.linalg import det 
from numpy.linalg import norm
from numpy.linalg import inv 

import math
from math import cos
from math import pi 
from math import sin 
from math import exp

In [42]:
def subspace(distribution):
    '''
    Get the boundaries of a subspace given a n x m distribution in n-dimensional space.

    Parameters
    ----------
    trajectory: np.array
        This is a n x m distribution in n-dimensional space over m samples
    
    Returns
    -------
    bounds: np.array
        This is a 2 x n array that descibes the limits in x_1, x_2, .. x_n dimensions. 
    '''
    #get the dimensions of the trajectory.
    n, m = distribution.shape

    #intitialize bounds to be a 2 x n matrix. 
    bounds = np.ndarray((2,n))

    #set the minimums on top and the maximums on the bottom.
    #round to nice even numbers
    bounds[0, :] = np.floor(np.amin(distribution, axis=1))
    bounds[1, :] = np.ceil(np.amax(distribution, axis=1))

    return bounds

In [43]:
#Distribution and Initialization of Constants---------------------------------
#creation of the distribution
mu = np.zeros((2))
covar= np.diag((2,2))
target = np.random.multivariate_normal(mu, covar, 100000).T
bounds = subspace(target)

#creation of our time boundary 
t = 10
timestep = 1000
time, dt = np.linspace(0,t,timestep,retstep=True)

#create the guess, trajectory, and controls
#initial control guess
cont_guess = np.vstack((np.full_like(time, 1), np.full_like(time, -1)))
traj_guess = np.zeros((2 * time.size))
traj_guess[0:2] = np.array([0, 1])

#update our trajectory based on our control signals
for i in range(0,len(traj_guess)-2, 2):
    #calculate the states based on the control velocities.
    ii = int(i/2)
    traj_guess[i + 2] = traj_guess[i]       + dt * cont_guess[0, ii]
    traj_guess[i + 3] = traj_guess[i + 1]   + dt * cont_guess[1, ii]

#reshape to represent states. 
traj_guess = traj_guess.reshape(time.size, 2).T

#initial guess
initial_guess = np.concatenate((traj_guess, cont_guess))
state_dict = dict(zip(time,initial_guess.T))

In [44]:
def fourier(k, L, x, n):
    '''
    Fourier will take the approximation of a singular n-dimensional k, using the basis function.

    Parameters
    ----------
    k : np.array
        This a singular n-dimensional index for Fourier coefficients.
    L: np.array
        Upper and lower bounds of an n-dimensional search space.
    x: np.array
        Trajectory point.
    n: int
        Number of dimensions in search space. 
    
    Returns
    -------
    prod: float
        Value of the Fourier Basis Function at that index, w.r.t trajectory point.
    '''
    #make sure that k is the same dimensionality as n 
    #also makes sure that k is a numpy array.
    if np.shape(k)[0] != n:
        raise TypeError("Wanted k to be np.array of size n")
    
    if np.shape(x)[0] != n:
        raise TypeError("Wanted x to be np.array of size n")
    
    if np.shape(L) != (2,n):
        raise TypeError("Wanted L to be np.array of size n")
    
    #find h_k. 
    h_k = 1 

    #find the product over the n-dimensional space based on the given
    #k and x(t)
    prod = 1 

    for i in range(n):
        prod *= cos((k[i] * pi)/(L[1, i] - L[0, i]) * (x[i] - L[0, i]))
    
    #return the product normalized by the h_k facotr.
    prod = 1/h_k * prod
    
    return prod

def phi_k(boundary, mu, cv, resolution, KF):
    '''
    Define phi_k over the bounds over a normally distributed Guassian distribution with 
    mu and covariance.

    Parameters
    ----------
    boundary: np.array
        This is a 2 x n distribution in n-dimensional space. Assumes [0,:] is minimums,
        [1,:] is maximums.

    mu: np.array
        This is a n-dimensional array of the Guassian distribution mean. 

    cv: np.array
        This is an n x n dimensional array of the Guassian distribution covariance.

    resolution: integer
        This tells us how fine our mesh should be in the distribution discretization    

    k_F: integer
        This tells us how many Fourier coefficients we will have.
    
    Returns
    -------
    pk: np.array
        This is a k_F by k_F dimensioned array of coefficients. 
    '''
    #set up our index arrays for the Fourier Coefficients.
    k = np.array([KF, KF])
    rows, cols = np.indices((k[0],k[1]))

    #intitialize the pk storage.
    pk = np.zeros((k[0],k[1]))

    #create the grid between the boundaries to the specified resolution.
    rangex, dx = np.linspace(boundary[0,0], boundary[1,0] + 1, resolution, retstep=True)
    rangey, dy = np.linspace(boundary[0,1], boundary[1,1] + 1, resolution, retstep=True)
    sub_x, sub_y = np.meshgrid(rangex, rangey)

    #iterate over our subspace indices.
    for ii in range(sub_x.shape[0]):        #number of rows
        for jj in range(sub_y.shape[1]):    #number of columns
            sub_index = np.array([sub_x[ii, jj], sub_y[ii, jj]])

            #find the px value for the given sub_index.
            px = det(2 * pi * cv)**-0.5 * exp(-0.5 * ((sub_index - mu).T) @ inv(cv) @ (sub_index - mu))

            #iterate over our fourier basis coefficient indices.
            for i in range(k[0]):
                for j in range(k[1]):

                    #fourier only accepts numpy arrays, so we convert into an index.
                    k_index = np.array([rows[i,j],cols[i,j]])

                    #find the pk matrix for the given timestep.         
                    pk[i,j] += px * fourier(k_index, boundary, sub_index, 2) * dy * dx

    return pk

In [45]:
#calculate the pk over the entire domain
pk = phi_k(bounds, mu, covar, 100, 10)

In [47]:
def metric(trajectory, bounds, time, dt, phi_k):
    '''
    Define the cost of the ergodicity of a trajectory.

    Parameters
    ----------
    trajectory: np.array
        This a n x m trajectory in n-dimensional space over m timesteps

    bounds: np.array
        This is a 2 x n distribution in n-dimensional space. Assumes [0,:] is minimums,
        [1,:] is maximums.

    time: np.array
        This is a m shaped array.
    
    dt: float
        Timestep.

    phi_k: np.array
        This is a k_F by k_F dimensioned array of coefficients. 

    q: float
        This tells us how much to weigh our ergodic metric in the cost function

    R: np.array
        Must be n x n dimensions. Weighs importance control signals.

    u: np.array
        This a n x m control array in n-dimensional space over m timesteps
    
    P1: np.array
        Must be n x n dimensions.
        
    Returns
    -------
    cost: float
        A measure of the current cost based on the ergodic metric.
    '''
    #create our ck array.
    n, m = phi_k.shape
    ck = np.zeros((n, m))

    #create an indexable array of Fourier Coefficients.
    k = np.array([n,m])
    rows, cols = np.indices((n, m))

    #find ck using Fourier Basis Functions
    for ii in range(trajectory.shape[1]):
        #iterate over the indices that we have.
        for i in range(n):
            for j in range(m):
                #fourier only accepts numpy arrays so we convert into an index.
                index = np.array([rows[i,j],cols[i,j]])

                #find the ck matrix for the given timestep
                ck[i, j] += fourier(index, bounds, trajectory[:, ii], 2) * dt

    #finsih the integral
    ck = 1/time[-1] * ck    

    #define the ergodic metric
    lamb = (1 + norm(k)**2)**(-1 * ((2 + 1) / 2)) 

    #define the difference between the ck matrix and the pk matrix
    diff = np.subtract(ck, phi_k)

    return diff

In [67]:
def DFourier(k, state, bounds):
    #we expect that k will be a np.array
    #we expect that state will be a np.array
    
    #unpack the bounds 
    k1, k2 = k[:]
    x, y = state[:]

    xmin, xmax = bounds[:,0]
    ymin, ymax = bounds[:,1]

    #define our h_k 
    h_k = 1 

    #create our storage space 
    val = np.ndarray((2,))

    #assign the values.
    val[0] = -1/h_k * sin((k1 * pi)/(xmax - xmin) * (x - xmin)) * cos((k2 * pi)/(ymax - ymin) * (y - ymin))  
    val[1] = -1/h_k * cos((k1 * pi)/(xmax - xmin) * (x - xmin)) * sin((k2 * pi)/(ymax - ymin) * (y - ymin))

    #return our vector
    return val

def DFArray(bounds, state, KF):
    #create our holding array
    DFK = np.zeros((KF, KF, 2))
    n, m, z = DFK.shape

    #create an indexable array of Fourier Coefficients 
    k = np.array([n,m])
    rows, cols = np.indices((n, m))

    #iterate over the indexes of the entire Fourier Coefficient Array
    for i in range(n):
        for j in range(m):
            #Dfourier only accepts numpy arrays so we convert into an index.
            index = np.array([rows[i,j],cols[i,j]])

            #find the DFK matrix for the given timestep
            DFK[i, j, :] += DFourier(index, state, bounds)
    return DFK

In [85]:
def A(t): 
    x, y, theta, u1, u2 = state_dict[t]
    return np.array([[0, 0 ],
                     [0, 0 ]
                     ]) 
                     
def B(t): 
    x, y, theta, u1, u2 = state_dict[t]
    return np.array([[1, 0],
                     [0, 1]
                     ]) 

def euler(P1, Q, R, t, dt):
    #Create the P array based on the time size.
    P = np.ndarray((4, t.size))

    #Assign the terminal condition to the end of the array. 
    P[:,-1] = P1.flatten()

    #Find the slope and work backwards using Euler's integration. 
    for i in range(t.size-1, 0, -1):
        #Find the current P matrix that we have and reshape it for matmul.
        cP = P[:,i].reshape(2,2)

        #Find the slope.
        Pdot = cP @ A(t[i]) + A(t[i]).T @ cP - cP @ B(t[i]) @ inv(R) @ B(t[i]).T @ cP + Q

        #Perform Euler's Inegration and update the next step.
        P[:, i-1] = (cP + Pdot * dt).flatten()

    return P

In [84]:
def a(t, q, diff, bounds, KF, T):
    #t is the time at which we are solving.
    #T is the time horizon

    #setup the state based on the given timestep.
    x, y, u1, u2 = state_dict[t]
    state = np.array([x, y])

    #setup the array that will be used to calculate our lambda
    k = np.array([KF, KF])
    lamb = (1 + norm(k)**2)**(-1 * ((2 + 1) / 2)) 

    #multiply the diff matrix elementwise by the matrix that we generate
    #by solving the differential Fourier Basis Functions
    DFk = DFArray(bounds, state, KF)
    DFk = 1/T * DFk

    #This is the elementwise multiplication
    DFk[:,:,0] = np.multiply( diff, DFk[:, :, 0])
    DFk[:,:,1] = np.multiply( diff, DFk[:, :, 1])

    #This is the summation over the pages of the matrix. Will output
    #a 2 x 1
    interior = q * np.sum(2 * lamb * DFk, axis=(0,1))
    return interior

def b(t, R):
    x, y, u1, u2 = state_dict[t]
    control = np.array([u1, u2])
    return control @ R 